In [37]:
import glob

path = "../api/index_documents"

files  = glob.glob(path + "/*")

#for filename in glob.glob(f"{path}/*"):
#    print(filename)

files

['../api/index_documents\\ED2_PAPER+ID+05+GENERATIVE+ART.pdf',
 '../api/index_documents\\genai-art.pdf',
 '../api/index_documents\\genai-music.pdf']

In [38]:
from langchain_community.document_loaders import PyPDFLoader

docs = []
for file in files:
    loader = PyPDFLoader(file)
    pages = loader.load()
    docs.append(pages)
    print(file)

../api/index_documents\ED2_PAPER+ID+05+GENERATIVE+ART.pdf
../api/index_documents\genai-art.pdf
../api/index_documents\genai-music.pdf


In [30]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.documents import Document
import os

embeddings = AzureOpenAIEmbeddings(
                azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                api_key=os.getenv('AZURE_OPENAI_KEY'),
                azure_deployment=os.getenv('EMBEDDINGS_DEPLOYMENT_NAME')
            )



#vector_store = FAISS.from_documents(docs[0], embeddings)

#vector_store.save_local(os.path.join(full_path, "faiss_index"))
#index_path = os.path.join(full_path, "faiss_index")

In [36]:
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

d = 1536
faiss_index = faiss.IndexFlatL2(d)

vs = FAISS( 
    embedding_function=embeddings,
    index=faiss_index,
    docstore= InMemoryDocstore(),
    index_to_docstore_id={}
    )

In [29]:
parent_dir = "../api/index"
def create_vector_store(documents):

        index = faiss.IndexFlatL2(1536)
        
        vector_store = FAISS(
            embedding_function=embeddings,
            index=index,
            docstore= InMemoryDocstore(),
            index_to_docstore_id={}
        )

        for document in documents:
            vector_store.add_documents(document)

        vector_store.save_local(os.path.join(parent_dir, "faiss_index"))
        return vector_store

In [39]:
vector_store = create_vector_store(docs)

In [40]:
vector_store.similarity_search("audio with gen ai", k=3)

[Document(id='0830b0aa-3515-4dd3-bd4f-1f1a9de0352d', metadata={'producer': 'pdfTeX-1.40.20; modified using iText 4.2.0 by 1T3XT', 'creator': 'LaTeX with hyperref', 'creationdate': '2020-01-09T07:59:40+00:00', 'keywords': 'Human-AI Interaction; Generative Deep Neural Networks; Co-Creation', 'moddate': '2025-02-26T13:02:18-08:00', 'trapped': '/False', 'subject': '', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019) kpathsea version 6.3.1', 'author': '', 'title': 'Novice-AI Music Co-Creation via AI-Steering Tools for Deep Generative Models', 'source': '../api/index_documents\\genai-music.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='CHI 2020 Paper\nCHI 2020, April 25–30, 2020, Honolulu, HI, USA\nNovice-AI Music Co-Creation via \nAI-Steering Tools for Deep Generative Models \nRyan Louie∗, Andy Coenen†, Cheng Zhi Huang, Michael Terry†, Carrie J. Cai† \n∗Northwestern University, †Google Research \nryanlouie@u.northwestern.edu, andycoen

In [19]:
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
parent_dir = "../api/index"

def create_vector_store(documents):

        index = faiss.IndexFlatL2(1536)

        vector_store = FAISS(
            embedding_function=embeddings,
            index=index,
            docstore= InMemoryDocstore(),
            index_to_docstore_id={}
        )

        for document in documents:
            vector_store.add_documents(document)

        vector_store.save_local(os.path.join(parent_dir, "faiss_index"))
        index_path = os.path.join(parent_dir, "faiss_index")
        return index_path

In [20]:
create_vector_store(docs)

'../api/index\\faiss_index'

In [11]:
index_path = "../api/index/faiss_index"
from langchain_community.vectorstores import FAISS

def query_index(query: str):
    vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    #retriever = vector_store.as_retriever()
    #documents = retriever.invoke(query)
    #result_string = "\n\n".join(doc.page_content for doc in documents)
    return vector_store

In [26]:
os.getcwd()+"\\index_documents"

'c:\\Users\\p.a.rodriguez.canedo\\Documents\\Hackaton_2025\\Notebooks\\index_documents'

In [21]:
vs = query_index("generative ai in art")

In [23]:
vs.similarity_search("generative ai")

[Document(id='8a061273-52b8-4f20-a1a5-0dd12f5390aa', metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-12-15T20:26:26+08:00', 'author': 'Teddy Soikun', 'moddate': '2023-12-15T20:26:26+08:00', 'source': '../api/index_documents\\ED2_PAPER+ID+05+GENERATIVE+ART.pdf', 'total_pages': 12, 'page': 1, 'page_label': '2'}, page_content="[59] \n \nproduce more advanced and higher-quality work and has helped visual artists produce art. AI \nis modernizing some processes, but it still requires human involvement to create something \nuseful and, most importantly, interesting. Therefore, AI can be seen as a tool that complements \nand enhances the work of designers and artists to make their work more productive and efficient. \nDesigners can fill the gap between human touch and creativity, which is fundamental to the \nartwork. Unlike computers, artists are endowe d with their own emotions and unique \nperspectives (Nigeria, 2023). \nGenerative art,

In [28]:
vs.index.ntotal

30